# Install Enviroment

In [7]:
import gym
env = gym.make("CartPole-v1")
observation, info = env.reset(seed=42)

for _ in range(1000):
    action = env.action_space.sample()
    observation, reward, terminated, truncated, info = env.step(action)

    if terminated or truncated:
        observation, info = env.reset()
env.close()

d:\download\python\Lib\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


In [10]:
pip install gym[box2d]

  Using cached box2d-py-2.3.5.tar.gz (374 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached pygame-2.1.0.tar.gz (5.8 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [77 lines of output]
      
      
      WARNING, No "Setup" File Exists, Running "buildconfig/config.py"
      Using WINDOWS configuration...
      
      Making dir :prebuilt_downloads:
      Downloading... https://www.libsdl.org/release/SDL2-devel-2.0.16-VC.zip 13d952c333f3c2ebe9b7bc0075b4ad2f784e7584
      Unzipping :prebuilt_downloads\SDL2-devel-2.0.16-VC.zip:
      Downloading... https://www.libsdl.org/projects/SDL_image/release/SDL2_image-devel-2.0.5-VC.zip 137f86474691f4e12e76e07d58d5920c8d844d5b
      Unzipping :prebuilt_downloads\SDL2_image-devel-2.0.5-VC.zip:
      Downloading... https://www.libsdl.org/projects/SDL_ttf/release/SDL2_ttf-devel-2.0.15-VC.zip 1436df41ebc47ac36e02ec9bda5699e80ff9bd27
      Unzipping :prebuilt_downloads\SDL2_ttf-devel-2.0.15-VC.zip:
      Downloading... https://www.libsdl.org/projects/SDL_mixer/release/SDL2_mixer-devel-2.0.4-VC.zi

In [12]:
import gymnasium as gym
gym.make('MountainCar-v0')

<TimeLimit<OrderEnforcing<PassiveEnvChecker<MountainCarEnv<MountainCar-v0>>>>>

In [ ]:
import gym
env = gym.make("LunarLander-v2", render_mode="human")
observation, info = env.reset(seed=42)
for _ in range(1000):
   action = policy(observation)  # User-defined policy function
   observation, reward, terminated, truncated, info = env.step(action)

   if terminated or truncated:
      observation, info = env.reset()
env.close()

In [1]:
import gym
from gym import spaces
import numpy as np

import math
from typing import Optional

import numpy as np

import gym
from gym import spaces
from gym.envs.classic_control import utils
from gym.error import DependencyNotInstalled

### Create Env

In [13]:
class BaseStationEnv(gym.Env):
    def __init__(self):
        super(BaseStationEnv, self).__init__()

        # Kích thước ma trận trạm base stand và tải
        self.matrix_size = 5

        # Không gian hành động: Bật/tắt trạm base stand (0: OFF, 1: ON)
        self.action_space = spaces.MultiDiscrete([2] * self.matrix_size ** 2)

        # Không gian quan sát: Ma trận tải
        self.observation_space = spaces.Box(0, 1, shape=(self.matrix_size, self.matrix_size), dtype=np.float32)

        # Trạng thái ban đầu
        self.state = np.full((self.matrix_size, self.matrix_size), 0.1, dtype=np.float32)

    def step(self, action):
        # Thực hiện hành động: Bật/tắt trạm base stand
        action_matrix = np.array(action).reshape((self.matrix_size, self.matrix_size))
        self.state = self.state * action_matrix

        # Tính thưởng
        energy_consumption = self.calculate_energy_consumption(self.state)
        traffic_loss = self.calculate_traffic_loss(self.state)
        reward = energy_consumption - 100 * traffic_loss

        # Xác định trạng thái kết thúc
        done = False

        # Trả về thông tin
        info = {}

        return self.state, reward, done, info

    def reset(self):
        # Khởi tạo trạng thái ban đầu
        self.state = np.full((self.matrix_size, self.matrix_size), 0.1, dtype=np.float32)
        return self.state

    def render(self, mode='human'):
        # Hiển thị trạng thái (tuỳ thuộc vào mode)
        if mode == 'human':
            print(self.state)
        elif mode == 'rgb_array':
            pass

    def calculate_energy_consumption(self, state):
        # Tính toán tiêu thụ năng lượng (theo công thức của bạn)
        return 0  # Cần triển khai công thức thực tế

    def calculate_traffic_loss(self, state):
        # Tính toán mất mát lưu lượng (theo công thức của bạn)
        return 0  # Cần triển khai công thức thực tế
